In [1]:


print(current_directory)
print(parent_directory)
print(processor_path)
print(controller_path)
print(queue_path)

%load_ext autoreload
%autoreload 2

/Users/wattjer/Desktop/quick_batch/notebooks
/Users/wattjer/Desktop/quick_batch
/Users/wattjer/Desktop/quick_batch/quick_batch/processor_app
/Users/wattjer/Desktop/quick_batch/quick_batch/controller_app
/Users/wattjer/Desktop/quick_batch/quick_batch/queue_app


In [31]:
import shutil
import os
import sys
import docker 


# path to this file's directory and parent directory
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# add to path
sys.path.append(parent_directory)

# define paths to each app dockerfile location 
processor_path = os.path.join(parent_directory, 'quick_batch', 'processor_app')
controller_path = os.path.join(parent_directory, 'quick_batch', 'controller_app')
queue_path = os.path.join(parent_directory, 'quick_batch', 'queue_app')


# create client for docker
def create_client():
    return docker.from_env()

# remove all docker containers
def remove_all_containers(client):
    for container in client.containers.list(all=True):
        container.remove(force=True)

# build processor_app docker image
def build_processor_image(client, processor_path):
    # copy processor requirements.txt to processor_app directory to roll into dockerfile
    shutil.copyfile(os.path.join(parent_directory, 'processor_requirements.txt'), os.path.join(processor_path, 'processor_requirements.txt'));

    # create docker image for processor app - including user defined requirements
    client.images.build(path=processor_path, tag='processor_app', quiet=False)

    # remove processor_requirements.txt from the processor_path directory 
    os.remove(os.path.join(processor_path, 'processor_requirements.txt'))

In [32]:
client = create_client()
build_processor_image(client, processor_path)

# remove_all_containers(client)

In [33]:
# start processor_app docker container with interactive terminal
processor_app_test = client.containers.run('processor_app', 
                                           detach=True, 
                                           name='processor_app', 
                                           tty=True, 
                                           stdin_open=True, 
                                           ports={'5000/tcp': 5000},
                                           volumes={processor_path + '/processor_app': 
                                               {'bind': '/processor_app', 'mode': 'rw'}});

In [36]:
# start process
processor_app_test

'processor_app'

In [ ]:
# execute hello_world.py in processor_app docker container
processor_app_test.exec_run('python3 -m processor_app.hello_world')

In [32]:
# list running docker containers by name
client.containers.list(all=True, filters={'name': 'processor_app'},)


TypeError: list() got an unexpected keyword argument 'status'

In [28]:
# list current docker images 
client.images.list()

In [ ]:
a